In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
type(df_train)

pandas.core.frame.DataFrame

In [4]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [5]:
df_train.shape

(75000, 4)

In [6]:
df_valid=df_test.copy()

In [7]:
df_train=df_train.loc[df_train['label']<2]

In [8]:
df_train.shape

(25000, 4)

In [9]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [10]:
type(df_train['tokens'])

pandas.core.series.Series

In [11]:
bs=52 # 52 - Jeremey, 20 - default

In [12]:
bptt= 70 #70 - Jeremey, 35 - default

In [13]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [14]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [15]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [17]:
padlen=1400
padding_idx=1

In [18]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [19]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [20]:
df_train.sort_values(by='n_tok', inplace=True)

In [21]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [22]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [23]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [24]:
df_train.shape

(25000, 5)

In [25]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [26]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [27]:
padlen

1400

In [28]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [29]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [30]:
padlen

1400

In [31]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [32]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [33]:
for xb,yb,xlen in dltrain:
    break

In [34]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
         1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
         1, 0, 1, 1]),
 tensor([106, 216,  56, 258, 293, 364, 157, 192, 206, 611, 360, 199, 231, 248,
         267, 199, 309, 620, 243, 135, 271, 311, 245, 167, 369, 212, 191, 160,
         130, 449, 159, 159, 223, 407, 108, 635, 465, 613, 201, 157, 340, 208,
         212, 247,  98, 746, 350, 490,  94, 193, 194,  75]))

## Model Architecture

In [35]:
from torch import nn
import torch
from torch.autograd import Variable

In [36]:
bs,bptt

(52, 70)

In [37]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-9
bidirectional=True
dropout_e=0.2 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [38]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [39]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [40]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [41]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [42]:
import torch

In [43]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [44]:
m(input).size()

torch.Size([1, 64, 5])

In [45]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters+self.n_hidden*4,self.n_out)
        

        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs)) 
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)
        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        
        ## Concatenate
        big_out=torch.cat([cat_cnn,hidden,max_pool],dim=1)
        preds=self.fc(big_out)

        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))

        
        return preds.view(-1),loss
        
    

In [46]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [47]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [48]:
device="cpu"

## Load pretrained model weights

In [49]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [50]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [51]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [52]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 30,883,801 trainable parameters


In [54]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=1900, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [55]:
model_sentiment.forward(xb,yb,xlen)

(tensor([-4.0499e-02,  3.4665e-01,  1.1984e-01,  5.2528e-01,  4.7953e-02,
          6.7727e-02,  1.1670e-01,  1.2763e-02,  3.3743e-01,  2.4670e-01,
          3.5124e-01,  4.6154e-01,  2.9537e-01,  1.9097e-01,  4.8709e-02,
         -5.0180e-02,  3.3347e-01, -5.3426e-02,  3.4331e-01,  1.3531e-01,
          2.8156e-01,  4.7378e-01,  6.2336e-02,  9.1662e-02,  1.0448e-01,
          1.3415e-02,  3.4778e-01,  3.3308e-01,  2.8071e-02,  1.1471e-01,
          1.9911e-01,  1.6053e-01,  3.1642e-01,  1.6062e-01,  1.7240e-04,
         -2.8532e-01,  5.2667e-02,  6.0814e-02,  3.4031e-02,  2.7878e-02,
          3.7732e-01,  6.3678e-01,  5.2455e-01, -2.8081e-02,  1.5299e-01,
          6.8433e-03,  2.7957e-01,  1.4266e-01,  1.0522e-01,  3.2672e-01,
          2.2065e-01,  1.0300e-01], grad_fn=<ViewBackward>),
 tensor(0.7115, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [56]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [57]:
preds.view(-1)

tensor([ 0.1125,  0.1972,  0.2212,  0.3236, -0.1584,  0.0467,  0.1999,  0.1609,
        -0.0416,  0.0421,  0.1293,  0.3338,  0.3305,  0.2429, -0.1384, -0.1017,
         0.4255, -0.0253,  0.0867, -0.0427,  0.4670,  0.0613,  0.1823,  0.1378,
         0.0476, -0.0602,  0.2966,  0.0135,  0.2490,  0.2271,  0.5776,  0.0766,
         0.2312,  0.3231,  0.0016, -0.0831,  0.2566,  0.1529,  0.0163, -0.0207,
         0.2636,  0.1314,  0.3900,  0.0597,  0.1504,  0.0089,  0.2234,  0.1629,
         0.2306, -0.1246,  0.1461, -0.0543], grad_fn=<ViewBackward>)

In [58]:
yb

tensor([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 1])

In [59]:
preds.size()

torch.Size([52])

In [60]:
yb.size()

torch.Size([52])

In [61]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.5769)

In [62]:
from sklearn.metrics import roc_auc_score

In [63]:
roc_auc_score(yb,preds)

0.46518518518518515

In [64]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [65]:
# Weight Decay Schedule
tempstart=1e-6
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*9

0 1e-06
1 9e-06
2 8.1e-05
3 0.000729
4 0.006561000000000001
5 0.059049000000000004


In [66]:
tempstart=5e-3
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*0.5

0 0.005
1 0.0025
2 0.00125
3 0.000625
4 0.0003125
5 0.00015625


In [67]:
torch.randint(1,5,(1,))

tensor([4])

In [68]:
lrs=[1e-2,5e-3,1e-4,5e-4]

In [69]:
torch.randint(0,4,(1,))

tensor([0])

In [70]:
lrs[torch.randint(0,4,(1,))]

0.005

In [71]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None,\
                 cycle_mult=0,lr_decay=0.5,wd_mult=9):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
        self.cycle_mult,self.lr_decay=cycle_mult,lr_decay
        self.wd_mult=wd_mult
        for param_group in self.optimizer.param_groups:
            self.start_lr=param_group['lr']
            self.start_wd=param_group['weight_decay']
        self.wd=self.start_wd
        self.lr=self.start_lr
        self.n_epoch=0
        self.lrs=[1e-2,5e-3,1e-4,5e-4]
        
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            if 1==0:
                lr =self.lrs[torch.randint(0,4,(1,))]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

     
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        
        if self.cycle_mult > 0:
            reset_cycle=self.cycle_mult
        
        for epoch in range(n_epochs):                

            self.n_epoch+=1
            loss,acc=self.run_epoch(dltrain,True)
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Learning rate {self.lr} Weight Decay {self.wd} Train Loss:{loss} Train Accuracy:{acc} Valid Loss:{lossv} Valid Accuracy:{accv}')
        
            if self.cycle_mult:
                if self.n_epoch==reset_cycle:
                    self.lr=self.start_lr
                    #self.wd=self.start_wd
                    reset_cycle=epoch+reset_cycle
                else:
                    self.lr*=self.lr_decay  
                    self.wd*=self.wd_mult
                
                
            for param_group in self.optimizer.param_groups:
                param_group['lr']=self.lr
                param_group['weight_decay']=self.wd

In [72]:
device="cuda:0"

In [73]:
n_layers

2

In [74]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [75]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
#optimizer=torch.optim.SGD(model_sentiment.parameters(),lr=1e-2,momentum=0.9, weight_decay=wd)
metric_fn=accuracy_binomial

In [76]:
len(dltrain),len(dlvalid)

(481, 481)

In [77]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [78]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,100,0.25,cycle_mult=0)

In [79]:
model_sentiment.freeze_embedding()

In [80]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [81]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [82]:
learner.run_epochs(dltrain,dlvalid,4)

Batch:100 0.6167932227253914  0.6721154093742371
Batch:200 0.4875051675736904  0.758653875887394
Batch:300 0.42137994075814883  0.799679519534111
Batch:400 0.39037883123382927  0.820913495272398
Batch:100 0.20247573107481004  0.9200000357627869
Batch:200 0.19995815943926573  0.9214423447847366
Batch:300 0.20420477618773777  0.9193590120474497
Batch:400 0.2059665451385081  0.9173077294230461
Epoch:0 Learning rate 0.005 Weight Decay 1e-09 Train Loss:0.36972531965951166 Train Accuracy:0.8330801560576393 Valid Loss:0.20816327072292753 Valid Accuracy:0.9164601358703169
Batch:100 0.22466189049184324  0.9094231182336807
Batch:200 0.2314952802658081  0.9051923483610154
Batch:300 0.2283688531567653  0.908782091140747
Batch:400 0.22267195102758705  0.9123077319562435
Batch:100 0.19813994392752649  0.9196154230833054
Batch:200 0.19234480425715447  0.9237500387430191
Batch:300 0.19599007247636716  0.9219231154521307
Batch:400 0.19653008421882986  0.9214423462748528
Epoch:1 Learning rate 0.005 Weig

In [101]:
learner.run_epochs(dltrain,dlvalid,1)

Epoch:0 Learning rate 0.005 Weight Decay 1e-07 Train Loss:0.21453498772455848 Train Accuracy:0.9136854701379232 Valid Loss:0.19502436239543428 Valid Accuracy:0.9237646309352961


In [102]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.924')
torch.save(optimizer.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.924')
torch.save (model_sentiment,f'{PATH}/inter/master_plannery_nounfreeze_0.924')
torch.save (optimizer,f'{PATH}/inter/master_plannery_nounfreeze_0.924')
torch.save (learner,f'{PATH}/inter/master_plannery_nounfreeze_0.924')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [103]:
model_sentiment.unfreeze_embedding()

In [104]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [105]:
learner.run_epochs(dltrain,dlvalid,2)

Epoch:0 Learning rate 0.005 Weight Decay 1e-07 Train Loss:0.20463219523553788 Train Accuracy:0.9198425150710678 Valid Loss:0.19661389088370448 Valid Accuracy:0.9238365974099125
Epoch:1 Learning rate 0.005 Weight Decay 1e-07 Train Loss:0.10808236364337176 Train Accuracy:0.9632856547212898 Valid Loss:0.2285887342248414 Valid Accuracy:0.910471012587359


In [179]:
len(dltrain)

433